In [3]:
import json
import requests
import os
import base64
import msal
import pandas as pd
import time

GRAPH_API_ENDPOINT = 'https://graph.microsoft.com/v1.0'
TENANT_ID = "bb3cbbb8-0b73-4ec6-b9e7-30ab4ad1ccfe"
AUTHORITY = f"https://login.microsoftonline.com/{TENANT_ID}"
CLIENT_ID = "c2f3f6c2-6b15-4990-94fb-fd7924122e4f"
scopes = ["https://graph.microsoft.com/.default"]

In [6]:
# Create a (preferably long-lived app instance) which maintains the token cache.
app = msal.PublicClientApplication(CLIENT_ID, client_credential=None, authority=AUTHORITY)

In [4]:
# Mail Parameter
# Subject
subject = "IGRM: Rechnung Mitgliedsbeitrag 2022"
# Body Text
#with open("mail-template.txt") as f:
#        bodytext = f.read() 
# Path to attachment
attachment_path = "C:/Users/jens-/Interessensgemeinschaft Records Management (IGRM)/Administration - Documents/2022/Finanzen Belege/Beitragsrechnungen Mitglieder/"
# Attachment filename
def attachment_filename(mitgliedsnummer):
        return f"IGRM_Mitgliedsrechnung_{mitgliedsnummer}.pdf"
# From address
from_address = "kassier@igrm.ch"
# Mail Text in HTML (Plain Text is derived automatically in MS365)
bodytext = """
<p>Guten Tag</p>
<p>Anbei erhalten Sie die Rechnung f&uuml;r den Mitgliedsbeitrag 2022 der Interessensgemeinschaft Records Management (IGRM). 
Auch dieses Jahr versenden wir die Rechnungen nur noch per Email.</p>
<p>Stimmt etwas nicht? Bitte melden Sie sich bei Problemen direkt bei mir.</p>
<p>Vielen Dank f&uuml;r ihre Treue zu unserem Verein.</p>
<p>Freundliche Gr&uuml;sse <br />Jens-Ole Petersen <br />Kassier IGRM <br />www.igrm.ch</p>
<p>---<br />
Jens-Ole Petersen <br />
Scheuerrain 5 <br />
3007 Bern <br />
jens-ole.petersen@igrm.ch <br />
kassier@igrm.ch <br />
079 667 16 88</p>
"""

In [5]:
# Listen Parameter
list_path = "C:/Users/jens-/Interessensgemeinschaft Records Management (IGRM)/Administration - Documents/2022/Finanzen/"
list_filename = "2022-11-24 Aktive Mitglieder zum Rechnungsversand.xlsx"

In [7]:
def send_email(message):
    result = None
    accounts = app.get_accounts()
    if accounts:
        result = app.acquire_token_silent(scopes, account=accounts[0])

    if not result:
        print("No suitable token exists in cache. Let's authenticate against Azure Active Directory.")
        result = app.acquire_token_interactive(scopes=scopes)

    if "access_token" in result:
        endpoint = f'https://graph.microsoft.com/v1.0/me/sendMail'
        email_msg = {'Message': message,
                    'SaveToSentItems': 'true'}
        r = requests.post(endpoint,
                          headers={'Authorization': 'Bearer ' + result['access_token'],
                                   "Content-Type": "application/json; charset=utf-8"}, 
                          json=email_msg)
        if r.ok:
            return True
        else:
            print(r.json())
            return False
    else:
        print("Das hat nicht geklappt. :-(")
        print(result.get("error"))
        print(result.get("error_description"))
        print(result.get("correlation_id"))
        return False

In [8]:
def get_attachment_object(mitgliedsnummer):

    file_full_path = "".join([attachment_path, attachment_filename(mitgliedsnummer)]) 

    if not os.path.exists(file_full_path):
        print('file is not found')
        return
    
    with open(file_full_path, 'rb') as upload:
        media_content = base64.b64encode(upload.read())
        
    attachment_object = {
        '@odata.type': '#microsoft.graph.fileAttachment',
        'contentType': "text/plain",
        'name': attachment_filename(mitgliedsnummer),
        'contentBytes': media_content.decode('utf-8')
    }
    return attachment_object

In [9]:
def get_message_object(mitgliedsnummer, to_email_adress):

    attachment = get_attachment_object(mitgliedsnummer)
    message_object = {  'Subject': subject,
                        'Body': {'ContentType': 'HTML', 
                                 'Content': bodytext},
                        'ToRecipients': [{'EmailAddress': {'Address': to_email_adress}}],
                        "from": {"emailAddress": {"address": from_address}},
                        'attachments': [ attachment ]
                        }
    return message_object

In [10]:
# Einzel Test
message = get_message_object(1006, "jens-ole.petersen@igrm.ch")
r = send_email(message)
if not r:
    print("STOP FEHLER")

No suitable token exists in cache. Let's authenticate against Azure Active Directory.


In [12]:
# Einlesen der Daten für den Lauf
liste = pd.read_excel(list_path + list_filename)    #csv: , sep=";"
rm_quote = lambda x: x.replace('"', '')
liste = liste.rename(columns=rm_quote)
liste = liste.set_index("ID")
liste = liste[["Nachname", "Vorname", "Mitgliedsnummer", "E-Mail Rechnungsversand"]]

In [14]:
for i in range(len(liste)):
#for i in range(3):

    row = liste.iloc[i]

    mitgliedsnr = row["Mitgliedsnummer"] 
    zieladresse = row["E-Mail Rechnungsversand"]
    #zieladresse = "jens-ole@petersen.top"
    print("Jetzt zu senden:", row["Nachname"], row["Vorname"], mitgliedsnr, zieladresse, row["E-Mail Rechnungsversand"])

    message = get_message_object(mitgliedsnr, zieladresse)
    result = send_email(message)
    if result:
        print("OK.")
    else:
        print("STOP FEHLER")
    
    time.sleep(3)  # Send Limit in O365 of 30 Mail per minute

Jetzt zu senden: Battaglia Francesco 1002 PDF-Rechnung@efv.admin.ch PDF-Rechnung@efv.admin.ch
OK.
Jetzt zu senden: Bazak Heike 1003 h.bazak@mfk.ch h.bazak@mfk.ch
OK.
Jetzt zu senden: Brenzikofer Theo 1004 theo.brenzikofer@gs-ejpd.admin.ch theo.brenzikofer@gs-ejpd.admin.ch
OK.
Jetzt zu senden: Brodbeck Beat 1005 beat.brodbeck@aredis.ch beat.brodbeck@aredis.ch
OK.
Jetzt zu senden: Bühler Rolf 1006 rolf.buehler@ndb.admin.ch rolf.buehler@ndb.admin.ch
OK.
Jetzt zu senden: Dessl Christoph 1007 christoph.dessl@kiwi-consultants.ch christoph.dessl@kiwi-consultants.ch
OK.
Jetzt zu senden: Haari Claudine 1008 PDF-Rechnung@efv.admin.ch PDF-Rechnung@efv.admin.ch
OK.
Jetzt zu senden: Hasler Raphael 1012 news@raphaelhasler.ch news@raphaelhasler.ch
OK.
Jetzt zu senden: Hofer Marc 1014 marc.hofer@archiventis.ch marc.hofer@archiventis.ch
OK.
Jetzt zu senden: Hofmann Roger 1015 hofmann-reist@bluewin.ch hofmann-reist@bluewin.ch
OK.
Jetzt zu senden: Järmann Stephan 1017 stephan.jaermann@swissmedic.ch steph